In [1]:
import ee
import geemap
import numpy as np
import time
# import geopandas as gpd

In [2]:
ee.Initialize()

In [3]:
#shp path
tp_crop ='crop_pt.shp'
tp_non_crop = 'non_crop_pt.shp'

#ee object
crop_ee = geemap.shp_to_ee(tp_crop)
noncrop_ee = geemap.shp_to_ee(tp_non_crop)
#feature collection
fc = crop_ee.merge(noncrop_ee)
#define bounding box
bbox = ee.Geometry.bounds(fc)
aoi = bbox
#TODO #read about system.index

In [4]:
sdate= '2020-04-01'
edate= '2020-08-31'

#cloud mask function
def clmask(image):
    qa = image.select('QA60')
    return image.updateMask(qa.lt(1)).divide(10000)


imgcollection = ee.ImageCollection("COPERNICUS/S2_SR")\
    .filterBounds(aoi)\
        .filterDate(sdate, edate)\
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))\
                .filter(ee.Filter.lt('CLOUD_COVERAGE_ASSESSMENT',5))\
                    .map(clmask)

image = imgcollection.median()

In [8]:
def Veg_Index(image_name):
    #NDVI
    ndvi = image_name.expression(
        '(NIR+RED)/(NIR - RED)', {
            'NIR': image_name.select('B8'),
            'RED': image_name.select('B4')
        }).rename('NDVI')
    #EVI
    evi = image_name.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',{
        'NIR': image_name.select('B8'),
        'RED': image_name.select('B4'),
        'BLUE': image_name.select('B2')
    }).rename('EVI')
    #GCVI
    gcvi = image_name.expression(
    '(NIR / GREEN) - 1',{
        'NIR':image_name.select('B8'),
        'GREEN':image_name.select('B3')
    }).rename('GCVI')

    return image_name.addBands(ndvi).addBands(evi).addBands(gcvi)

In [9]:
bb = Veg_Index(image)

In [11]:
#get band properties -https://gis.stackexchange.com/questions/395579/blank-image-pansharpening-sentinel-2-imagery-in-google-earth-engine
band3 = image.select('B3')
proj = band3.projection().getInfo()
crs = proj['crs']
#resample the resolution
band20m = image.select('B8A', 'B11', 'B12')
swir_b10m = band20m.resample('bilinear').reproject(crs=crs, scale=10.0).rename('B8Ar','B11r', 'B12r')
image = image.addBands(swir_b10m)


#Vegetation Index 
#other VI to consider GNDVI - https://pro.arcgis.com/en/pro-app/latest/arcpy/image-analyst/gnvdi.htm
def Veg_Index(image_name):
    #NDVI
    ndvi = image_name.expression(
        '(NIR+RED)/(NIR - RED)', {
            'NIR': image_name.select('B8'),
            'RED': image_name.select('B4')
        }).rename('NDVI')
    #EVI
    evi = image_name.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',{
        'NIR': image_name.select('B8'),
        'RED': image_name.select('B4'),
        'BLUE': image_name.select('B2')
    }).rename('EVI')
    #GCVI
    gcvi = image_name.expression(
    '(NIR / GREEN) - 1',{
        'NIR':image_name.select('B8'),
        'GREEN':image_name.select('B3')
    }).rename('GCVI')

    return image_name.addBands(ndvi).addBands(evi).addBands(gcvi)
with_vi = Veg_Index(image)    

#select bands needed for the classification including NDVI
band_sl = ('B2', 'B3', 'B4', 'B8', 'B8Ar', 'B11r', 'B12r', 'NDVI', 'GCVI', 'EVI' )
#dataset for classification
image_feature = with_vi.select(band_sl)

image_feature = image_feature.clip(aoi)

In [12]:
#exprot to assest - Need review\not working
task = ee.batch.Export.image.toAsset(**{'image':image_feature,
'description': 'Compositeband',
'assetId':'users/aadebowaledaniel/cpclasscomposite', 
'scale': 10, 
'region':aoi.getInfo()['coordinates'],
'maxPixels': 1e13,
})
task.start()
#Monitor the task
while task.status()['state'] in ['READY', 'RUNNING']:
  print(task.status())
  time.sleep(5)
else:
  print(task.status())

EGSYW27Y3SUI'}
{'state': 'RUNNING', 'description': 'Compositeband', 'creation_timestamp_ms': 1620784938141, 'update_timestamp_ms': 1620785399357, 'start_timestamp_ms': 1620784958958, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'FP4VZGG3MHGFEGSYW27Y3SUI', 'name': 'projects/earthengine-legacy/operations/FP4VZGG3MHGFEGSYW27Y3SUI'}
{'state': 'RUNNING', 'description': 'Compositeband', 'creation_timestamp_ms': 1620784938141, 'update_timestamp_ms': 1620785459352, 'start_timestamp_ms': 1620784958958, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'FP4VZGG3MHGFEGSYW27Y3SUI', 'name': 'projects/earthengine-legacy/operations/FP4VZGG3MHGFEGSYW27Y3SUI'}
{'state': 'RUNNING', 'description': 'Compositeband', 'creation_timestamp_ms': 1620784938141, 'update_timestamp_ms': 1620785459352, 'start_timestamp_ms': 1620784958958, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'FP4VZGG3MHGFEGSYW27Y3SUI', 'name': 'projects/earthengine-legacy/operations/FP4VZGG3MHGFEGSYW27Y3SUI'}
{'state': 'RUNNING', 'des

In [17]:
#Monitor the task
while task.status()['state'] in ['READY', 'RUNNING']:
  print(task.status())
  time.sleep(5)
else:
  print(task.status())

{'state': 'COMPLETED', 'description': 'Compositeband', 'creation_timestamp_ms': 1620202099413, 'update_timestamp_ms': 1620202758608, 'start_timestamp_ms': 1620202115006, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://code.earthengine.google.com/?asset=projects/earthengine-legacy/assets/users/aadebowaledaniel/cbinary'], 'attempt': 1, 'id': 'GOSO2UFWSS2GSUKMXFOFFSMQ', 'name': 'projects/earthengine-legacy/operations/GOSO2UFWSS2GSUKMXFOFFSMQ'}


In [4]:
#import composite from asset
predimage = ee.Image('users/aadebowaledaniel/cbinary')

In [5]:
predbands = predimage.bandNames()

In [6]:
#continue from here
trainingSample  = predimage.sampleRegions(
    collection = fc,
    properties=['label'],
    scale = 10, 
    geometries = True
)
# trainingSample.size().getInfo()

rantraning = trainingSample.randomColumn('random')
split = 0.7
trainsample = rantraning.filter(ee.Filter.lt('random', split))
testsample = rantraning.filter(ee.Filter.gte('random', split))

#train random forest classifier
classifier = ee.Classifier.smileRandomForest(100).train(trainsample, 'label', predbands)


#confusion matrix train Accurracy
trainAccuracy = classifier.confusionMatrix()
print('Resubstitution error matrix: ', trainAccuracy.getInfo())
print('Training overall accuracy: ', trainAccuracy.accuracy().getInfo())

#classify features
testclassifier = testsample.classify(classifier)
#cconfusion matrix representing expected accuracy
testAccuracy = testclassifier.errorMatrix('label', 'classification')
print('confusion Matrix', testAccuracy.getInfo())
print('Valdation overall accuracy:', testAccuracy.accuracy().getInfo())

#classify image
Imageclassified = predimage.classify(classifier)
# altImageclassified = predimage.classify(classifier)

Resubstitution error matrix:  [[463, 8], [0, 1790]]
Training overall accuracy:  0.9964617425917736
confusion Matrix [[136, 60], [25, 776]]
Valdation overall accuracy: 0.9147442326980942


In [7]:
Map = geemap.Map(center=(0.9613,34.6536),zoom=8)
Map

Map(center=[0.9613, 34.6536], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [8]:
Map.addLayer(predimage, {'bands': ['B4', 'B3', 'B2'], 'min': 0.0, 'max': 0.3, 'gamma': 2}, 'True colour composite')

In [11]:
cropmask = Imageclassified.eq(1)
# vis = {'pallete':['#18d848']}
Map.addLayer(cropmask,{'color':'green'}, 'Vegetation')

cropmask = Imageclassified.eq(2)

cropmaskdd = cropmask.updateMask(cropmask)

cropmask_vec = cropmaskdd.reduceToVectors(
    geometryType= 'polygon',
    reducer= ee.Reducer.countEvery(),
    scale= 10,
    maxPixels= 1e13,
    geometry= aoi
)
Map.addLayer(cropmask_vec, {'color':'green'}, 'Crop Mask')

In [15]:
# #convert to Vector
# cropmask = Imageclassified.eq(1)

# vector_cropmask = cropmask.updateMask(cropmask).reduceToVectors(
#     reducer=ee.Reducer.countEvery(),
#     geometry= aoi,
#     scale=10,
#     maxPixels=1e13)
# def sim(nna):
#     return nna.simplify(maxError = 100)
# vector_cropmask = vector_cropmask.map(sim)
# cropmaskVec = cropmask.reduceToVectors({
#     # 'geometryType': 'polygon',
#   'reducer': ee.Reducer.countEvery(),
#   'scale': 10,
#   'geometry':aoi,
# })

# vector_img = Imageclassified.unmask(0).reduceToVectors(
#         geometry = aoi,
#         scale = 10,
#         maxPixels = 1e13,
#         bestEffort =True
#     )
# taskvec = ee.batch.Export.table.toAsset(collection=vector_cropmask,
# description= 'Vegetation mask', assetId= 'users/aadebowaledaniel/vegetationmask')
# taskvec.start()
# #Monitor the task
# while taskvec.status()['state'] in ['READY', 'RUNNING']:
#   print(taskvec.status())
#   time.sleep(5)
# else:
#   print(taskvec.status())

In [13]:
# vis_params = {'pallete':['E5FFCC',
#             'D8D8D8',
#             ]}
# #{'min':0,'max':1,'palette':["green","white"]}            
# Map.addLayer(Imageclassified,{}, 'Classified')